# Import a finetuned Qwen3 model from HuggingFace into Amazon Bedrock CMI, using SageMaker JupyterLab environment

Custom Model Import (CMI) is a feature that allows you to import model artifacts of certain model architectures to Amazon Bedrock and run them in a **serverless way**. For more information on CMI check the [Bedrock documentation on CMI](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-import-model.html).

## The challenge
---
In order for CMI to work, model artifacts need to be on an S3 bucket. The size of model artifacts nowadays can be quite large (e.g. exceeding 100GB) which can create issues with downloading them locally. In this notebook we will show how to download large Qwen 3 model artifacts from HuggingFace to JupyterLab environment in SageMaker Studio, and import them to Bedrock using CMI, in order to have serverless access to different types of Qwen 3 models. The same approach can be generalized for any other model artifacts from all the supported families in CMI. 

We will be focusing on the following steps: 

- Configuring SageMaker Studio and the JupyterLab environment, in order to allow receiving large model artifacts.
- Downloading the model artifacts from HuggingFace locally to JupyterLab.
- Uploading the model artifacts to S3.
- Initiating a CMI job.
- Performing inference on the imported model.

You can find more details in this AWS Blogpost: [Deploy Qwen models with Amazon Bedrock Custom Model Import](https://aws.amazon.com/blogs/machine-learning/deploy-qwen-models-with-amazon-bedrock-custom-model-import)

## Prerequisites
---
In order to run this notebook, you need the right IAM permissions. Specifically:
- The notebook's execution role needs to be able to access Amazon Bedrock (and specifically all CMI-related actions) and have a trusted relationship with Amazon Bedrock (in order for it to be able to assume this role and carry out CMI operations). 
- You need appropriate Permissions to access model files in Amazon S3. 
- You need a S3 bucket to store the custom model artifacts (later to be imported to Bedrock using CMI).
- Sufficient local storage space (this is what we will be setting up next).

In order to set the correct IAM roles and permissions, follow [the instruction here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-import-iam-role.html) . 
The following are some policy examples that you can consider. Substitue the `AWS_REGION`, `ACCOUNT_ID`, `BUCKET_NAME` and `IAM_ROLE_ARN` variables with the ones relevant for your case, from the role that you will create.

**Policy for allowing access to CMI-related tasks**: Your notebook role should have these following minimum permissions to allow CMI tasks.

```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "BedrockInvokeModelAndStreaming",
      "Effect": "Allow",
      "Action": [
        "bedrock:InvokeModel",
        "bedrock:InvokeModelWithResponseStream"
      ],
      "Resource": [
        "arn:aws:bedrock:{AWS_REGION}:{ACCOUNT_ID}:imported-model/*",
        "arn:aws:bedrock:{AWS_REGION}:{ACCOUNT_ID}:custom-model-deployment/*"
      ]
    },
    {
      "Sid": "BedrockCustomModelImportJobManagement",
      "Effect": "Allow",
      "Action": [
        "bedrock:CreateModelImportJob",
        "bedrock:ListModelImportJobs",
        "bedrock:GetModelImportJob"
      ],
      "Resource": "*"
    },
    {
      "Sid": "BedrockImportedModelLifecycle",
      "Effect": "Allow",
      "Action": [
        "bedrock:ListImportedModels",
        "bedrock:GetImportedModel",
        "bedrock:DeleteImportedModel"
      ],
      "Resource": [
        "arn:aws:bedrock:{AWS_REGION}:{ACCOUNT_ID}:imported-model/*"
      ]
    },
    {
       "Sid": "AccessModelFilesInS3",
       "Effect": "Allow",
       "Action": [
         "s3:GetObject",
         "s3:ListBucket",
         "s3:PutObject"
       ],
       "Resource": [
         "arn:aws:s3:::{BUCKET_NAME}",
         "arn:aws:s3:::{BUCKET_NAME}/*"
       ],
       "Condition": {
         "StringEquals": {
         "aws:ResourceAccount": "{ACCOUNT_ID}"
        }
      }
    },
    {
       "Sid": "PassRoleToBedrockForCMI",
       "Effect": "Allow",
       "Action": "iam:PassRole",
       "Resource": "{IAM_ROLE_ARN}",
       "Condition": {
         "StringEquals": {
         "iam:PassedToService": "bedrock.amazonaws.com"
         }
       }
    }
  ]
}
```


**Trust relationship**: The following policy allows Amazon Bedrock to assume this role and carry out model import operations. Include the following policy to your IAM Role's Trusted Relationships. 

```
{
    "Version":"2012-10-17",		 	 	 
    "Statement": [
        {
            "Sid": "1",
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": "{ACCOUNT_ID}"
                },
                "ArnEquals": {
                    "aws:SourceArn": "arn:aws:bedrock:{AWS_REGION}:{ACCOUNT_ID}:model-import-job/*"
                }
            }
        }
    ]
}
```


## Setting up the environment
---

We need to have a large local space in order to save the model artifacts that will be downloaded from HuggingFace. For example, for Qwen 3 30B the model artifacts are around 122GB. By default, JupyterLab and SageMaker Studio Domains, don't allow for such large available spaces. You need to explicitly configure them to make this happen. In this section we will be performing the following steps: 

- Enabling a large EBS size in your SageMaker Studio Domain.
- Creating a JupyterLab space with adequate EBS storage.
- Making this storage available to store the HF model artifacts.

Configure your SageMaker Studio Domain to allow large EBS storage spaces, by setting up a large storage space under "Edit storage settings". Here you need to select an adequate number to cover the size of the model artifacts. In the following image we have selected 500GB.

![Alt text](images/sm-studio-domain.png)

You need also to create a new JupyterLab space (from within SageMaker Studio) and allow adequate storage for JupyterLab. The maximum storage you can select in this step, is bounded by the maximum number you have set in your Studio Domain configuration.

![Alt text](images/sm-studio-jupyterlab-space.png)

Within the notebook, inside the JupyterLab environment, you also need to set the required environmental variables that will allow you to download the model artifacts from HuggingFace and safe them within the local file system. 

In [ ]:
import os

# Put all temporary + cache files on the large 500GB volume
os.environ["HF_HOME"] = "/home/sagemaker-user/hf_cache"
os.environ["HF_HUB_CACHE"] = "/home/sagemaker-user/hf_cache"
os.environ["HF_DATASETS_CACHE"] = "/home/sagemaker-user/hf_cache"
os.environ["TMPDIR"] = "/home/sagemaker-user/tmp"

# Disable the Xet CAS download mechanism
os.environ["HF_HUB_ENABLE_HFS_CAS"] = "0"
os.environ["HF_HUB_DISABLE_XET"] = "1"

os.makedirs("/home/sagemaker-user/hf_cache", exist_ok=True)
os.makedirs("/home/sagemaker-user/tmp", exist_ok=True)

In [ ]:
# constants
import sagemaker

HF_REPO_ID = "<huggingface-repo-id>"       # change this with your relevant repo id location in HuggingFace, e.g. prakhag/qwen3-30b-ft
MODEL_ARTIFACTS = "llm_artifacts"          # this is the local directory that will store the model artifacts from HuggingFace
S3_BUCKET_ARTIFACTS = "<your-s3-bucket>"   # change this to the name of your S3 bucket that will store the model artifacts
S3_PREFIX = "qwen3-artifacts"              # change this to a prefix name in your S3 bucket, where all the model artifacts will be stored
CMI_JOB_NAME = "import-Qwen3"              # Job names need to be unique in your account and region, so if you rerun, you need to change to a new job name
IMPORTED_MODEL_NAME = "Qwen3-imported"     # Imported model names need to be unique in your account and region, so if you rerun, you need to change to a new model name
CMI_ROLE = sagemaker.get_execution_role()  # the current role of the notebook or any other IAM role you want
print("IAM role to be used for the CMI job:", CMI_ROLE)

## Download the model artifacts from HuggingFace
---
Now we are ready to download the model artifacts in the JupyterLab's local file system. You will need to point to a specific HuggingFace repo for a particular model. In the following cell subtitute the `repo_id` with your relevant repo in HuggingFace. 

In [ ]:
# upgrading the huggingface_hub package
!pip install --upgrade --no-cache-dir huggingface_hub

In [ ]:
# download model artifacts from HF locally to JupyterLab 
# this may take some time depending on the size of the model artifacts and your network connection (e.g. 15-20min for ~130GB)

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id=HF_REPO_ID,
    local_dir=f"/home/sagemaker-user/{MODEL_ARTIFACTS}",
    local_dir_use_symlinks=False
)

## Upload model artifacts to S3 and initiate a CMI job
---
We will now upload the model artifacts from the local folder to S3. This operation make take some time, depending on the size of the model artifacts and your network connection. 

In [ ]:
import boto3

s3 = boto3.client("s3")

def upload_directory_to_s3(local_dir, bucket, prefix):
    """
    Recursively uploads a local directory to S3 under the specified prefix.
    Preserves relative folder structure exactly.
    """
    local_dir = os.path.abspath(local_dir)

    for root, dirs, files in os.walk(local_dir):
        for filename in files:
            local_path = os.path.join(root, filename)
            # compute relative path within the directory
            relative_path = os.path.relpath(local_path, local_dir)
            s3_key = f"{prefix}/{relative_path}"

            print(f"Uploading: {local_path} → s3://{bucket}/{s3_key}")

            s3.upload_file(local_path, bucket, s3_key)

    print("Upload complete.")


upload_directory_to_s3(
    local_dir=MODEL_ARTIFACTS, 
    bucket=S3_BUCKET_ARTIFACTS, 
    prefix=S3_PREFIX
)

We are ready now to initiate a CMI job, that will ingest the model artifacts from S3 and will prepare a model ready to serve requests. CMI jobs may take a few minutes, depending on the size of the model artifacts. As an indication, for a Qwen3 30B parameters (130GB model artifacts) it takes around 15min. 

If you haven't set the IAM permissions correctly, as indicated in the Prerequsites section, the CMI will fail. You can find the exact reason of why the job failed by looking at the Amazon Bedrock Console -> "Imported Models" -> "Jobs" tab. Clich on the (failed) job and you will see the reason of failing, which will indicate which part of the IAM policy you need to fix. 

Also, there is a limited number of imported models that you can have in a region on an AWS account. If you receive a quote exceed exception, you will either need to delete some of your other imported models, or increase your quotas for the number of imported models. 

In [ ]:
import time

# Initialize the Bedrock client
bedrock = boto3.client('bedrock')

s3_uri = f"s3://{S3_BUCKET_ARTIFACTS}/{S3_PREFIX}/"

# Create the model import job
response = bedrock.create_model_import_job(
    jobName=CMI_JOB_NAME,
    importedModelName=IMPORTED_MODEL_NAME,
    roleArn=CMI_ROLE,  # or any other IAM role you want
    modelDataSource={
        's3DataSource': {
            's3Uri': s3_uri
        }
    }
)

job_Arn = response['jobArn']

# Output the job ARN
print(f"Model import job created with ARN: {response['jobArn']}")

# Check CMI job status
while True:
    response = bedrock.get_model_import_job(jobIdentifier=job_Arn)
    status = response['status'].upper()
    print(f"Status of import job: {status}")
    
    if status in ['COMPLETED', 'FAILED']:
        break
        
    time.sleep(30)  # Check every 30 seconds

# Get the model ID
model_id = response['importedModelArn']

## Testing the imported model
---
**IMPORTANT**: (as of today: January 2026) for Qwen3 models, `Converse` API is **not supported**. Only the `Invoke` API is supported! For more information please take a look at the [CMI documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-import-model.html).

In [ ]:
import json

client = boto3.client("bedrock-runtime", region_name="us-east-1")

def call_invoke_model_and_print(request):
    body = json.dumps(request).encode("utf-8")
    response = client.invoke_model(
        modelId=model_id,
        body=body,
        accept="application/json",
        contentType="application/json"
    )

    model_response = json.loads(response["body"].read())

    # robust field selection
    response_text = (
        model_response.get("generation")
        or model_response.get("output_text")
        or model_response.get("generated_text")
        or model_response.get("text")
    )

    # print(response_text)
    return response_text

**IMPORTANT**: there will be a cold start period, counting from the time of the first model invocation. During that time, you will receive a `ModelNotReadyException` error. This may take a few minutes, until the model is "hot" and ready to serve requests. Try the following request a few times until the model is ready to respond. 

In [ ]:
prompt = "What is the meaning of life?"

request = {
    "prompt": prompt,
    "max_gen_len": 1000,
    "top_p": 0.9,
    "temperature": 0.2,
}

call_invoke_model_and_print(request)

If the model is not invoked for 5 minutes, then it will become "cold". After that, any subsequent request will trigger the cold start period where the model will become ready again to surve requests. 